In [ ]:
!pip install tensorflow
!pip install torch torchvision
!pip install numpy
!pip install matplotlib
!pip install skimage

In [ ]:
# Use this to include your google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
cd drive/MyDrive/DL\ project

In [ ]:
cd "src/DL project"

In [ ]:
import numpy as np
from PIL import Image
import os
import pickler
from sklearn.neighbors import NearestNeighbors
from skimage.io import imread, imshow
from skimage.color import rgb2lab, lab2rgb
from skimage.util import crop
import matplotlib.pyplot as plt
import math

In [ ]:
path = 'data/images_64_batch'
path1 = 'data/images_batch'
path_32 = "data/images_small_batch"
path_256 = "data/images_256_batch"

path_cifar = "data/data_batch_1"

In [ ]:

# Download the 313 Q (ab paris) if it doesn't exist
!wget -P "data/" -nc https://github.com/junyanz/interactive-deep-colorization/raw/master/data/color_bins/pts_in_hull.npy
ab_pairs = np.load("data/pts_in_hull.npy")

def softencode(image_lab):
  # inspiration taken from https://github.com/foamliu/Colorful-Image-Colorization/blob/f498f5de1b505b056673ccca09fbb133c9e81ed1/data_generator.py#L15
  h, w = image_lab.shape[0], image_lab.shape[1]
  
  # Form ab pairs
  a = image_lab[:, :, 1].flatten()
  b = image_lab[:, :, 2].flatten()
  ab = np.vstack((a,b)).T
  
  # Get the nearest neighbours and their distances
  nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(ab_pairs)
  dist, idx = nn.kneighbors(ab)

  # Normalize distance using Gaussian kernel
  weights = np.exp(-(dist**2)/(2 * (5 ** 2)))
  weights = weights / np.sum(weights, axis=1, keepdims=True)

  y = np.zeros((ab.shape[0], ab_pairs.shape[0]))
  idx_pts = np.arange(ab.shape[0]).reshape(-1, 1)

  y[idx_pts, idx] = weights
  y = y.reshape(h, w, ab_pairs.shape[0])
  return y

def decode(predicted, im):
  h, w = predicted.shape[0], predicted.shape[1]
  a = ab_pairs[:,0].reshape((1, -1))
  b = ab_pairs[:,1].reshape((1, -1))

  T = 0.38
  predicted = predicted.reshape((h * w, -1))
  X = np.exp(np.log(predicted + 1e-8) / T)

  X = X / np.sum(X, axis=1, keepdims=True)
  Xa = np.sum(X * a, 1).reshape((h, w))
  Xb = np.sum(X * b, 1).reshape((h, w))

  out = np.zeros((h, w, 3), dtype=np.float32)
  out[:, :, 0] = im[:, :, 0]
  out[:, :, 1] = Xa
  out[:, :, 2] = Xb
  return out

def load_to_data(path, ratio):
  images = pickler.unpickle(path)
  #data = np.asarray(images)[:600].reshape(-1, 3, 32, 32)
  #data = np.transpose(data, (0, 2, 3, 1))
  data = []
  for image in images[0:math.floor(ratio*len(images))]:
    data.append(np.asarray(image))
  
  del images

  data_lab = []
  for d in data:
    data_lab.append(rgb2lab(d, illuminant='D50'))

  del data

  X = []
  Y = []
  for d in data_lab:
    h, w = d.shape[0], d.shape[1]
    #L = d[:,:,0].flatten()
    x = d[:,:,0].flatten()
    x = x.reshape(h,w,1)
    #y = softencode(d)
    y = d
    X.append(x)
    Y.append(y)

  return X,Y

def load_upsize(path, ratio):
  images = pickler.unpickle(path)
  data = []

  for image in images[0:math.floor(ratio*len(images))]:
    image = image.resize([256,256],Image.NEAREST)
    data.append(np.asarray(image))

  data_lab = []
  for d in data: 
    D = d.reshape(256,256,3)
    data_lab.append(rgb2lab(D, illuminant='D50'))

  X = []
  Y = []
  for d in data_lab:
    h, w = d.shape[0], d.shape[1]
    L = d[:,:,0].flatten()
    x = L
    x = x.reshape(h,w,1)
    y = softencode(d)
    X.append(x)
    Y.append(y)

  return X,Y


In [ ]:
# 64x64
# X,Y = load_to_data(path, 0.008)
# 32x32
X, Y = load_to_data(path_32, 0.03)
print(len(X), len(Y))
print(X[0].shape, Y[0].shape)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F


class model_original(nn.Module):
    def __init__(self):
        super(model_original, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=True)  #output size 
        self.conv1_1 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1, bias=True)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True) #output size 
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1, bias=True)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv3_1 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1, bias=True)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv4_1 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True)
        self.conv6 = nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True)
        self.conv6_1 = nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True)
        self.conv6_2 = nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True)
        self.conv7  = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv7_1  = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv7_2  = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv8 = nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=True)
        self.conv8_1 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv8_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv8_end = nn.Conv2d(256, 313, kernel_size=1, stride=1, padding=0, bias=True)


        self.batch1 = nn.BatchNorm2d(64)
        self.batch2 = nn.BatchNorm2d(128)
        self.batch3 = nn.BatchNorm2d(256)
        self.batch4 = nn.BatchNorm2d(512)
        self.soft = nn.Softmax(dim=1)
        self.model_out = nn.Conv2d(313, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=False)
        self.upsample4 = nn.Upsample(scale_factor=4, mode='bilinear', align_corners=False)

    def forward(self, x):
        x = self.batch1(F.relu(self.conv1_1(F.relu(self.conv1(x))))) #block1
        x = self.batch2(F.relu(self.conv2_2(F.relu(self.conv2(x))))) #block2
        x = self.batch3(F.relu(self.conv3_2(F.relu(self.conv3_1(F.relu(self.conv3(x))))))) #block3
        x = self.batch4(F.relu(self.conv4_2(F.relu(self.conv4_1(F.relu(self.conv4(x))))))) #block4
        x = self.batch4(F.relu(self.conv5_2(F.relu(self.conv5_1(F.relu(self.conv5(x))))))) #block5
        x = self.batch4(F.relu(self.conv6_2(F.relu(self.conv6_1(F.relu(self.conv6(x))))))) #block6
        x = self.batch4(F.relu(self.conv7_2(F.relu(self.conv7_1(F.relu(self.conv7(x))))))) #block7
        x = self.conv8_end(F.relu(self.conv8_2(F.relu(self.conv8_1(F.relu(self.conv8(x))))))) #block8
        #out = self.model_out(self.soft(x))
        out = self.soft(x)

        return self.upsample4(out)
      
class model_32(nn.Module): #with 32*32, the size are the same after each block
    def __init__(self):
        super(model_32, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=2, bias=True)  #output size 
        self.conv1_1 = nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2, bias=True) #output size 
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv3_1 = nn.Conv2d(256, 256, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv4_1 = nn.Conv2d(512, 512, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv5 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv6 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv6_1 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv6_2 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv7  = nn.Conv2d(512, 512, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv7_1  = nn.Conv2d(512, 512, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv7_2  = nn.Conv2d(512, 512, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv8 = nn.ConvTranspose2d(512, 256, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv8_1 = nn.Conv2d(256, 256, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv8_2 = nn.Conv2d(256, 256, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv8_end = nn.Conv2d(256, 313, kernel_size=5, stride=1, padding=2, bias=True)


        self.batch1 = nn.BatchNorm2d(64)
        self.batch2 = nn.BatchNorm2d(128)
        self.batch3 = nn.BatchNorm2d(256)
        self.batch4 = nn.BatchNorm2d(512)
        self.soft = nn.Softmax(dim=1)
        self.model_out = nn.Conv2d(313, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=False)
        self.upsample4 = nn.Upsample(scale_factor=4, mode='bilinear', align_corners=False)

    def forward(self, x):
        x = self.batch1(F.relu(self.conv1_1(F.relu(self.conv1(x))))) #block1
        x = self.batch2(F.relu(self.conv2_2(F.relu(self.conv2(x))))) #block2
        x = self.batch3(F.relu(self.conv3_2(F.relu(self.conv3_1(F.relu(self.conv3(x))))))) #block3
        x = self.batch4(F.relu(self.conv4_2(F.relu(self.conv4_1(F.relu(self.conv4(x))))))) #block4
        x = self.batch4(F.relu(self.conv5_2(F.relu(self.conv5_1(F.relu(self.conv5(x))))))) #block5
        x = self.batch4(F.relu(self.conv6_2(F.relu(self.conv6_1(F.relu(self.conv6(x))))))) #block6
        x = self.batch4(F.relu(self.conv7_2(F.relu(self.conv7_1(F.relu(self.conv7(x))))))) #block7
        x = self.conv8_end(self.batch3(F.relu(self.conv8_2(F.relu(self.conv8_1(F.relu(self.conv8(x)))))))) #block8
        #out = self.model_out(self.soft(x))
        out = self.soft(x)

        return self.upsample4(out)
    
    def predict(self, image):
      
      pass


class model_32_reduced(nn.Module): #with 32*32, the size are the same after each block
    def __init__(self):
        super(model_32_reduced, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=2, bias=True)  #output size
        torch.nn.init.kaiming_normal_(self.conv1.weight)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2, bias=True) #output size 
        torch.nn.init.kaiming_normal_(self.conv2.weight)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2, bias=True)
        torch.nn.init.kaiming_normal_(self.conv3.weight)

        #self.conv4 = nn.Conv2d(256, 512, kernel_size=5, stride=1, padding=2, bias=True)
        #self.conv7  = nn.Conv2d(512, 512, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv8 = nn.ConvTranspose2d(256, 256, kernel_size=5, stride=1, padding=2, bias=True)
        torch.nn.init.kaiming_normal_(self.conv8.weight)

        self.conv8_end = nn.Conv2d(256, 313, kernel_size=5, stride=1, padding=2, bias=True)
        torch.nn.init.kaiming_normal_(self.conv8_end.weight)


        self.batch1 = nn.BatchNorm2d(64)
        self.batch2 = nn.BatchNorm2d(128)
        self.batch3 = nn.BatchNorm2d(256)
        self.batch4 = nn.BatchNorm2d(512)
        self.soft = nn.Softmax(dim=1)
        self.model_out = nn.Conv2d(313, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=False)
        torch.nn.init.kaiming_normal_(self.model_out.weight)
        self.upsample4 = nn.Upsample(scale_factor=4, mode='bilinear', align_corners=False)

    def forward(self, x):
        x = self.batch1(F.relu(self.conv1(x))) #block1
        x = self.batch2(F.relu(self.conv2(x))) #block2
        x = self.batch3(F.relu(self.conv3(x))) #block3
        #x = self.batch4(F.relu(self.conv4(x))) #block4
        #x = self.batch4(F.relu(self.conv7(x))) #block7
        x = self.conv8_end(self.batch3(F.relu(self.conv8(x))))#block8
        #out = self.model_out(self.soft(x))
        out = self.soft(x)

        return out
    
    def predict(self, image):
      
      pass

class model_64(nn.Module): #with 64*64 input and pooling layers
    def __init__(self):
        super(model_64, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=True)  #output size 
        self.conv1_1 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.pool1 = nn.MaxPool2d(kernel_size = 2, stride=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True) #output size 
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True)
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv3_1 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv4_1 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv5 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv6 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv6_1 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv6_2 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv7  = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv7_1  = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv7_2  = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv8 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv8_1 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv8_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv8_end = nn.Conv2d(256, 313, kernel_size=3, stride=1, padding=1, bias=True)


        self.batch1 = nn.BatchNorm2d(64)
        self.batch2 = nn.BatchNorm2d(128)
        self.batch3 = nn.BatchNorm2d(256)
        self.batch4 = nn.BatchNorm2d(512)
        self.soft = nn.Softmax(dim=1)
        self.model_out = nn.Conv2d(313, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=False)
        self.upsample4 = nn.Upsample(scale_factor=8, mode='bilinear', align_corners=False)

    def forward(self, x):
        x = self.batch1(F.relu(self.conv1_1(F.relu(self.conv1(x))))) #block1
        x = self.batch2(F.relu(self.conv2_2(F.relu(self.conv2(x))))) #block2
        x = self.batch3(F.relu(self.conv3_2(F.relu(self.conv3_1(F.relu(self.conv3(x))))))) #block3
        x = self.batch4(F.relu(self.conv4_2(F.relu(self.conv4_1(F.relu(self.conv4(x))))))) #block4
        x = self.batch4(F.relu(self.conv5_2(F.relu(self.conv5_1(F.relu(self.conv5(x))))))) #block5
        x = self.batch4(F.relu(self.conv6_2(F.relu(self.conv6_1(F.relu(self.conv6(x))))))) #block6
        x = self.batch4(F.relu(self.conv7_2(F.relu(self.conv7_1(F.relu(self.conv7(x))))))) #block7
        x = self.conv8_end(self.batch3(F.relu(self.conv8_2(F.relu(self.conv8_1(F.relu(self.conv8(x)))))))) #block8
        out = self.model_out(self.soft(x))
        #out = self.soft(x)

        return self.upsample4(out)
    
    def predict(self, image):
      
      pass


class model_64_nopool(nn.Module): #with 64*64 input and pooling layers
    def __init__(self):
        super(model_64_nopool, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=True)  #output size 
        self.conv1_1 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1, bias=True)
        #self.pool1 = nn.MaxPool2d(kernel_size = 2, stride=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True) #output size 
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True)
        #self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv3_1 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv4_1 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv5 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv6 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv6_1 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv6_2 = nn.Conv2d(512, 512, kernel_size=5, dilation=2, stride=1, padding=2, bias=True)
        self.conv7  = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv7_1  = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv7_2  = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv8 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv8_1 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv8_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv8_end = nn.Conv2d(256, 313, kernel_size=3, stride=1, padding=1, bias=True)


        self.batch1 = nn.BatchNorm2d(64)
        self.batch2 = nn.BatchNorm2d(128)
        self.batch3 = nn.BatchNorm2d(256)
        self.batch4 = nn.BatchNorm2d(512)
        self.soft = nn.Softmax(dim=1)
        self.model_out = nn.Conv2d(313, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=False)
        self.upsample4 = nn.Upsample(scale_factor=8, mode='bilinear', align_corners=False)

    def forward(self, x):
        x = self.batch1(F.relu(self.conv1_1(F.relu(self.conv1(x))))) #block1
        x = self.batch2(F.relu(self.conv2_2(F.relu(self.conv2(x))))) #block2
        x = self.batch3(F.relu(self.conv3_2(F.relu(self.conv3_1(F.relu(self.conv3(x))))))) #block3
        x = self.batch4(F.relu(self.conv4_2(F.relu(self.conv4_1(F.relu(self.conv4(x))))))) #block4
        x = self.batch4(F.relu(self.conv5_2(F.relu(self.conv5_1(F.relu(self.conv5(x))))))) #block5
        x = self.batch4(F.relu(self.conv6_2(F.relu(self.conv6_1(F.relu(self.conv6(x))))))) #block6
        x = self.batch4(F.relu(self.conv7_2(F.relu(self.conv7_1(F.relu(self.conv7(x))))))) #block7
        x = self.conv8_end(self.batch3(F.relu(self.conv8_2(F.relu(self.conv8_1(F.relu(self.conv8(x)))))))) #block8
        #out = self.model_out(self.soft(x))
        out = self.soft(x)

        return self.upsample4(out)
    
    def predict(self, image):
      
      pass

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model_32_reduced()
model.to(device)

In [ ]:
learning_rate = 0.01
num_epochs = 120
batch_size = 50
import torch.optim as optim

# create a stochastic gradient descent optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-3)

# create a loss function
#criterion = nn.CrossEntropyLoss()

def multinomial_cross_entropy_loss(y_true, y_pred):
    y_true = softencode(y_true)
    y_true = torch.from_numpy(y_true).float().to(device) 

    idx_max = torch.argmax(y_true, -1).reshape(-1)
    prior_t = torch.from_numpy(prior_factor).float().to(device)

    weights = torch.gather(prior_t, 0, idx_max).reshape(32, 32)

    y_pred = torch.clamp(y_pred, 1e-8, 1-1e-8)
    sum = torch.sum(y_true * y_pred.log(), -1)
    
    return -1 * torch.sum(weights * sum)

In [ ]:
#import tensorflow as tf
#from torch.autograd import Variable
#cce = tf.keras.losses.CategoricalCrossentropy()
def ker_loss(y_true, y_pred):
    y_true = softencode(y_true)
    y_pred = unnormalize_label(y_pred).cpu().detach().numpy()
    loss = torch.tensor(cce(y_true, y_pred).numpy())
    loss = Variable(loss, requires_grad = True)
    return loss.to(device)


In [ ]:
ab_pairs = np.load("data/pts_in_hull.npy")

In [ ]:
# Adapted from https://github.com/foamliu/Colorful-Image-Colorization/blob/master/class_rebal.py

## Prior Probs
Xab = np.asarray(Y[:])[:, :, :, 1:]

print(Xab.shape)

Xa = np.ravel(Xab[:, :, :, 0])
Xb = np.ravel(Xab[:, :, :, 1])
X_ab = np.vstack((Xa, Xb)).T

# Get the nearest neighbours and their distances
nn = NearestNeighbors(n_neighbors=1, algorithm='kd_tree').fit(ab_pairs)
dist, idx = nn.kneighbors(X_ab)

ind = np.ravel(idx)
counts = np.bincount(ind)
idxs = np.nonzero(counts)[0]

prior_prob = np.zeros((ab_pairs.shape[0]))

prior_prob[idxs] = counts[idxs]

# We turn this into a color probability
prior_prob = prior_prob / (np.sum(prior_prob))
np.save("priors.npy", prior_prob)

In [ ]:
## Smoooth the prior probs

from scipy.signal import gaussian, convolve
from scipy.interpolate import interp1d

prior_prob += 1E-3 * np.min(prior_prob)
# renormalize
prior_prob = prior_prob / (1.0 * np.sum(prior_prob))

# Smooth with gaussian
f = interp1d(np.arange(prior_prob.shape[0]), prior_prob)
xx = np.linspace(0, prior_prob.shape[0] - 1, 1000)
yy = f(xx)
window = gaussian(2000, 5)
smoothed = convolve(yy, window / window.sum(), mode='same')
fout = interp1d(xx, smoothed)
prior_prob_smoothed = np.array([fout(i) for i in range(prior_prob.shape[0])])
prior_prob_smoothed = prior_prob_smoothed / np.sum(prior_prob_smoothed)

# Save
np.save("prior_prob_smoothed.npy", prior_prob_smoothed)

In [ ]:
u = np.ones_like(prior_prob_smoothed)
u = u / np.sum(1.0 * u)

prior_factor = (1 - 0.5) * prior_prob_smoothed + 0.5 * u
prior_factor = np.power(prior_factor, -1)

# renormalize
prior_factor = prior_factor / (np.sum(prior_factor * prior_prob_smoothed))

np.save("prior_factor.npy", prior_factor)
print(prior_factor.shape)

In [ ]:
#Normalize in the same way as the original paper

data_cent = 60
data_norm = 100
label_norm = 110

def normalize_data(data):
  return (data-data_cent)/data_norm

def unnormalize_data(data):
  return data_norm*data+data_cent

def normalize_label(label):
  return label/label_norm

def unnormalize_label(label):
  return label*label_norm



#split train&test
train_data = X[0:math.floor(len(X)*0.8)]
train_label = Y[0:math.floor(len(X)*0.8)]
test_data = X[math.floor(len(X)*0.8):]
test_label = Y [math.floor(len(X)*0.8):]
print(train_data[0].shape)
#convert to numpy arrays and transpose into correct order to CNN
train_data = np.asarray(train_data)
train_label = np.asarray(train_label)
test_data = np.asarray(test_data)
test_label = np.asarray(test_label)
print(train_data.shape)
train_data = np.transpose(train_data,(0,3,1,2) )
train_label = np.transpose(train_label,(0,3,1,2) )
test_data = np.transpose(test_data,(0,3,1,2) )
test_label = np.transpose(test_label,(0,3,1,2) )
print(train_data.shape)
train_data = normalize_data(train_data)
#train_label = normalize_label(train_label) # Normalize labels after softencoding in loss function
test_data = normalize_data(test_data)
#test_label = normalize_label(test_label) # Normalize labels after softencoding in loss function

print(train_data.shape)
#Send data to device
train_images = torch.from_numpy(train_data).float().to(device)
#train_labels = torch.from_numpy(train_label).float().to(device)


In [ ]:
for epoch in range(1, num_epochs+1):
  per = torch.randperm(train_images.shape[0])
  if epoch % 5 == 0 or epoch == 1:
    # Show decoded image
    H, W = train_images.shape[-1], train_images.shape[-2]
    train_im = np.random.randint(0, 500)
    train_example = train_images[train_im]
    train_l = np.transpose(train_label[train_im], (1, 2, 0))

    train_example = train_example.reshape(1, 1, H, W)
    train_pred = model(train_example).reshape([H, W, 313])
    train_pred = decode(train_pred.cpu().detach().numpy(), train_l)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True)
    ax1.imshow(lab2rgb(train_l))
    ax1.set_title("Original")
    ax2.imshow(lab2rgb(train_pred))
    ax2.set_title("Decoded:")
    plt.show()

  for i in range(0, train_images.shape[0], batch_size):
    
    ind = per[i:i+batch_size]
    batch_x, target_batch_y = train_images[ind,:,:,:], train_label[ind,:,:,:]
    
    torch.cuda.current_stream().synchronize()
    optimizer.zero_grad()
    output = model(batch_x)

    # Supposed to be 32 x 32 x 313
    out_t = torch.reshape(output, (ind.shape[0], 32, 32, 313))
    #out_t = torch.reshape(output, (ind.shape[0], 256, 256, 313))

    # Supposed to be 32 x 32 x 3
    true_t = np.transpose(target_batch_y, (0, 2, 3, 1))
    #true_t = torch.from_numpy(true_t).to(device)
    
    loss = 0
    for j in range(ind.shape[0]):
      loss += multinomial_cross_entropy_loss(true_t[j], out_t[j])
    loss /= ind.shape[0]

    #loss = criterion(true_t, out_t)

    loss.backward()
    optimizer.step()
    

    print("\rIteration:", i, "/", train_images.shape[0], "Loss:", loss.item(), end="")

  print("\rEpoch:", epoch, "/", num_epochs, "Loss:", loss.item(), end="\n")

torch.save(model, "small_model_0.1")


In [ ]:
del train_images

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import matplotlib.pyplot as plt

### Put test inputs onto GPU
test_images = torch.from_numpy(test_data).float().to(device)
H, W = test_images.shape[-1], test_images.shape[-2]
print(H, W)

### Pick one test example
test_number = np.random.randint(0, 100)
test_example = test_images[test_number]
test_l = np.transpose(test_label[test_number], (1, 2, 0))

### Pass the test input into the model
# Reshape input for CNN
test_example = test_example.reshape(1, 1, H, W)

prediction = model(test_example).reshape([H, W, 313]).cpu().detach().numpy()
pred_decoded = decode(prediction, test_l)

print(np.min(test_l[:,:,1:]), np.max(test_l[:, :, 1:]))
print(np.min(pred_decoded[:,:,1:]), np.max(pred_decoded[:, :, 1:]))

### Plot the test input
im = test_example.cpu().numpy().reshape(H, W)
plt.imshow(lab2rgb(test_l))
plt.show()

### Plot the prediction
plt.imshow(lab2rgb(pred_decoded))
plt.show()

In [ ]:
dummy = np.zeros((H, W, 1))
dummy[:,:,0] = 50

In [ ]:
pred1 = prediction

In [ ]:
pred2 = prediction

In [ ]:
print(np.sum(pred1[0,0]), np.max(pred1))
print(np.sum(pred2), np.max(pred2))

In [ ]:
plt.imshow(lab2rgb(decode(pred1, dummy)))
plt.show()

plt.imshow(lab2rgb(decode(pred2, dummy)))
plt.show()

In [ ]:
#t = (pred1 - pred2)**2
#print(np.min(t), np.max(t))

In [ ]:
### CONVERT 32 x 32 to 256 x 256
#data = []
#images = pickler.unpickle(path1)
#for image in images:
  #image = image.resize([256,256], Image.NEAREST)
  #image = image.resize([64,64], Image.NEAREST)
  #data.append(np.asarray(image))

#pickler.inpickle("data/images_256_batch", data)
#pickler.inpickle("data/images_64_batch", data)

In [ ]:
#model_path = "small_model_0.1"
#model = torch.load(model_path)
#model.eval()
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model.to(device)